## Load Packages

In [1]:
#using Dates       #Julia 0.7
include("printmat.jl")

printTable (generic function with 10 methods)

In [3]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# MV Frontier  with Two Assets

MV analysis starts with inputs of the vector of expected returns $\mu$ and the covariance matrix $\Sigma$.


Recall: with only two investable assets, all portfolios of them are on the MV frontier. We can trace out the entire MV frontier by calculating the means and standard deviations of a range of portfolios with different weights ($w_1,1-w_1$) on the two assets.

To calculate the expected return and the variance of a portfolio, use $ER_p = w'\mu$ and $Var(R_p) = w'\Sigma w$.

In [4]:
μ = [0.115; 0.06]            #means
Σ  = [166   58;              #covariance matrix
       58  100]/100^2    

println("expected returns: ")
printmat(μ)
println("covariance matrix:")
printmat(Σ)

expected returns: 
     0.115
     0.060

covariance matrix:
     0.017     0.006
     0.006     0.010



In [5]:
w₁ = 1.5:-0.05:-0.5           #different possible weights on asset 1
n  = length(w₁)

ERp   = fill(NaN,n)
StdRp = fill(NaN,n)
for i = 1:n
    local w                #w is local to the loop
    w        = [w₁[i];1-w₁[i]]
    ERp[i]   = w'μ
    StdRp[i] = sqrt(w'Σ*w)
end   

In [6]:
plot2 = plot(StdRp*100,ERp*100,legend=nothing,color=:red,xlim=(0,15),ylim=(0,15))
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip7902)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 674.886,47.9273 656.491,53.9521 638.331,59.9769 620.428,66.0016 602.807,72.0264 585.496,78.0512 568.525,84.076 551.928,90.1008 535.744,96.1256 520.014,102.15 
 504.786,108.175 490.111,114.2 476.044,120.225 462.648,126.249 449.99,132.274 438.14,138.299 427.173,144.324 417.168,150.349 408.204,156.373 400.36,162.398 
 393.712,168.423 388.328,174.448 384.269,180.473 381.584,186.497 380.304,192.522 380.447,198.547 382.009,204.572 384.973,210.596 389.301,216.621 394.942,222.646 
 401.836,228.671 409.91,234.696 419.088,240.72 429.292,246.745 440.441,252.77 452.459,258.795 465.271,264.82 478.806,270.844 492.999,276.869 507.789,282.894 
 523.122,288.919 
 "/>

# Calculating the MV Frontier: 3 or More (Risky) Assets

In [7]:
μ = [0.115; 0.095; 0.06]    #expected returns
Σ  = [166  34  58;              #covariance matrix
       34  64   4;
       58   4 100]/100^2                  
Rf = 0.03

println("μ, Σ and Rf: ")
printmat(μ)
printmat(Σ)
printmat(Rf)

μ, Σ and Rf: 
     0.115
     0.095
     0.060

     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010

     0.030



All portfolios on the MV frontier of risky assets only have (a vector of) portfolio weights as in

$w = \Sigma^{-1}(\mu \lambda + \mathbf{1} \delta)$, 

where the scalars $\lambda$ and $\delta$ depend on $(\mu,\Sigma)$ and the required average return $\mu^*$.

Some of the intermediate calculations are as follows. (Later on we put it all in a function)

In [8]:
mustar = 0.1                          #required average return  

n    = length(μ)
oneV = ones(n) 
 
Σ_1  = inv(Σ)

println("inv(Σ):")
printmat(Σ_1)
println("Σ*inv(Σ):")
printmat(Σ*Σ_1)

A    = μ'Σ_1*μ                  
B    = μ'Σ_1*oneV
C    = oneV'Σ_1*oneV
printlnPs("A, B, C: ",[A B C ])

λ    = (C*mustar - B)/(A*C-B^2)
δ    = (A-B*mustar)/(A*C-B^2)
printlnPs("λ and δ: ",[λ δ])

w    = Σ_1 *(μ*λ + oneV*δ)
println("w: ")
printmat(w)

inv(Σ):
    85.735   -42.545   -48.024
   -42.545   177.754    17.566
   -48.024    17.566   127.151

Σ*inv(Σ):
     1.000     0.000     0.000
     0.000     1.000     0.000
    -0.000     0.000     1.000

A, B, C:      1.804    19.759   244.634
λ and δ:      0.093    -0.003
w: 
     0.288
     0.690
     0.022



In [9]:
function MVCalc(mustar,μ,Σ) #the std of a portfolio on MVF of risky assets
    n    = length(μ)
    oneV = ones(n) 
    Σ_1  = inv(Σ)
    A    = μ'Σ_1*μ                 
    B    = μ'Σ_1*oneV
    C    = oneV'Σ_1*oneV
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt(w'Σ*w)
    return StdRp,w
end

MVCalc (generic function with 1 method)

In [10]:
(StdAt10,wAt10) = MVCalc(0.1,μ,Σ)
printlnPs("Just testing: std and w of portfolio with mean return 10: ",StdAt10)
printmat(wAt10)

Just testing: std and w of portfolio with mean return 10:      0.077
     0.288
     0.690
     0.022



In [11]:
mustar = linspace(Rf,0.15,101)          
L      = length(mustar)
StdRp  = fill(NaN,L)                      #loop over different required average returns, (mustar)
for i = 1:L
    StdRp[i] = MVCalc(mustar[i],μ,Σ)[1]   #[1] picks out only the first function output
end

In [12]:
plot3 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlim=(0,15),ylim=(0,15))
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip8102)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 503.328,294.396 495.201,291.767 487.126,289.138 479.103,286.509 471.137,283.88 463.23,281.251 455.386,278.622 447.608,275.993 439.901,273.364 432.268,270.735 
 424.715,268.106 417.246,265.477 409.865,262.848 402.58,260.219 395.395,257.59 388.316,254.961 381.352,252.332 374.508,249.703 367.793,247.074 361.214,244.445 
 354.781,241.816 348.502,239.187 342.387,236.558 336.448,233.929 330.694,231.3 325.137,228.671 319.789,226.042 314.662,223.413 309.77,220.784 305.126,218.155 
 300.742,215.526 296.633,212.897 292.812,210.268 289.293,207.639 286.089,205.01 283.213,202.381 280.676,199.752 278.49,197.123 276.665,194.494 275.209,191.865 
 274.13,189.236 273.432,186.607 273.119,183.978 273.193,181.349 273.653,178.72 274.497,176.091 275.721,173.462 277.319,170.833 279.283,168.204 281.604,165.575 
 284.272,162.946 287.275,160.317 290.601,157.688 294.237,155.059 298.169,152.43 302.385,149.801 306.87,147.172 311.611,144.543 316.594,141.914 321.807,139.285 
 327.236,136.656 332.87,134.027 338.696,131.398 344.704,128.769 350.882,126.14 357.221,123.511 363.711,120.882 370.343,118.253 377.108,115.624 383.999,112.995 
 391.008,110.366 398.128,107.737 405.352,105.108 412.674,102.479 420.089,99.85 427.591,97.221 435.176,94.592 442.837,91.963 450.572,89.334 458.375,86.705 
 466.244,84.076 474.174,81.447 482.162,78.818 490.205,76.189 498.301,73.56 506.445,70.931 514.637,68.302 522.873,65.673 531.151,63.044 539.469,60.415 
 547.825,57.786 556.217,55.157 564.644,52.528 573.103,49.899 581.594,47.27 590.114,44.641 598.663,42.012 607.239,39.3831 615.841,36.7541 624.467,34.1251 
 633.117,31.4961 
 "/>

# Calculating the MV Frontier (of Risky and Riskfree Assets)

All portfolios on the MV frontier of both risky and riskfree have (a vector of) portfolio weights on the risky assets as in 

$w=\frac{\mu^{\ast}-R_{f}}{(\mu^{e})^{\prime}\Sigma^{-1}\mu^{e}}\Sigma^{-1}
\mu^{e}$, 

where $\mu^*$ is the required average return.

The weight of the riskfree asset is $1-\mathbf{1}'w$

In [13]:
function MVCalcRf(mustar,μ,Σ,Rf)           #calculates the std of a portfolio on MVF of (Risky,Riskfree)
    n     = length(μ)
    μe    = μ .- Rf                        #expected excess returns            
    Σ_1   = inv(Σ)
    w     = (mustar-Rf)/(μe'Σ_1*μe) * Σ_1*μe
    StdRp = sqrt(w'Σ*w)
    return StdRp,w
end

MVCalcRf (generic function with 1 method)

In [14]:
StdRpRf  = fill(NaN,L)                 #loop over required average returns (mustar)
for i = 1:L
    StdRpRf[i] = MVCalcRf(mustar[i],μ,Σ,Rf)[1]
end    

In [15]:
plot4 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlim=(0,15),ylim=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip8302)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 503.328,294.396 495.201,291.767 487.126,289.138 479.103,286.509 471.137,283.88 463.23,281.251 455.386,278.622 447.608,275.993 439.901,273.364 432.268,270.735 
 424.715,268.106 417.246,265.477 409.865,262.848 402.58,260.219 395.395,257.59 388.316,254.961 381.352,252.332 374.508,249.703 367.793,247.074 361.214,244.445 
 354.781,241.816 348.502,239.187 342.387,236.558 336.448,233.929 330.694,231.3 325.137,228.671 319.789,226.042 314.662,223.413 309.77,220.784 305.126,218.155 
 300.742,215.526 296.633,212.897 292.812,210.268 289.293,207.639 286.089,205.01 283.213,202.381 280.676,199.752 278.49,197.123 276.665,194.494 275.209,191.865 
 274.13,189.236 273.432,186.607 273.119,183.978 273.193,181.349 273.653,178.72 274.497,176.091 275.721,173.462 277.319,170.833 279.283,168.204 281.604,165.575 
 284.272,162.946 287.275,160.317 290.601,157.688 294.237,155.059 298.169,152.43 302.385,149.801 306.87,147.172 311.611,144.543 316.594,141.914 321.807,139.285 
 327.236,136.656 332.87,134.027 338.696,131.398 344.704,128.769 350.882,126.14 357.221,123.511 363.711,120.882 370.343,118.253 377.108,115.624 383.999,112.995 
 391.008,110.366 398.128,107.737 405.352,105.108 412.674,102.479 420.089,99.85 427.591,97.221 435.176,94.592 442.837,91.963 450.572,89.334 458.375,86.705 
 466.244,84.076 474.174,81.447 482.162,78.818 490.205,76.189 498.301,73.56 506.445,70.931 514.637,68.302 522.873,65.673 531.151,63.044 539.469,60.415 
 547.825,57.786 556.217,55.157 564.644,52.528 573.103,49.899 581.594,47.27 590.114,44.641 598.663,42.012 607.239,39.3831 615.841,36.7541 624.467,34.1251 
 633.117,31.4961 
 "/>
<polyline clip-path="url(#clip8302)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8772,294.396 49.5555,291.767 54.2338,289.138 58.9121,286.509 63.5905,283.88 68.2688,281.251 72.9471,278.622 77.6254,275.993 82.3037,273.364 86.982,270.735 
 91.6603,268.106 96.3386,265.477 101.017,262.848 105.695,260.219 110.374,257.59 115.052,254.961 119.73,252.332 124.408,249.703 129.087,247.074 133.765,244.445 
 138.443,241.816 143.122,239.187 147.8,236.558 152.478,233.929 157.157,231.3 161.835,228.671 166.513,226.042 171.191,223.413 175.87,220.784 180.548,218.155 
 185.226,215.526 189.905,212.897 194.583,210.268 199.261,207.639 203.94,205.01 208.618,202.381 213.296,199.752 217.975,197.123 222.653,194.494 227.331,191.865 
 232.009,189.236 236.688,186.607 241.366,183.978 246.044,181.349 250.723,178.72 255.401,176.091 260.079,173.462 264.758,170.833 269.436,168.204 274.114,165.575 
 278.792,162.946 283.471,160.317 288.149,157.688 292.827,155.059 297.506,152.43 302.184,149.801 306.862,147.172 311.541,144.543 316.219,141.914 320.897,139.285 
 325.576,136.656 330.254,134.027 334.932,131.398 339.61,128.769 344.289,126.14 348.967,123.511 353.645,120.882 358.324,118.253 363.002,115.624 367.68,112.995 
 372.359,110.366 377.037,107.737 381.715,105.108 386.394,102.479 391.072,99.85 395.75,97.221 400.428,94.592 405.107,91.963 409.785,89.334 414.463,86.705 
 419.142,84.076 423.82,81.447 428.498,78.818 433.177,76.189 437.855,73.56 442.533,70.931 447.211,68.302 451.89,65.673 456.568,63.044 461.246,60.415 
 465.925,57.786 470.603,55.157 475.281,52.528 479.96,49.899 484.638,47.27 489.316,44.641 493.995,42.012 498.673,39.3831 503.351,36.7541 508.029,34.1251 
 512.708,31.4961 
 "/>

# Tangency Portfolio

The tangency portfolio is a particular portfolio on the MV frontier of risky and riskfree, where the weights on the risky assets sum to one. It is therefore also on the MV frontier of risky assets only. The vector of portfolio weights is

$w_{T}=\frac{\Sigma^{-1}\mu^{e}}{\mathbf{1}^{\prime}\Sigma^{-1}\mu^{e}}$

In [16]:
function MVTangencyP(μ,Σ,Rf)           #calculates the tangency portfolio
    n     = length(μ)
    oneV  = ones(n)  
    μe    = μ .- Rf                    #expected excess returns            
    Σ_1   = inv(Σ)
    w     = Σ_1 *μe/(oneV'Σ_1*μe)
    muT   = w'μ + (1-sum(w))*Rf
    StdT = sqrt(w'Σ*w)
    return w,muT,StdT
end

MVTangencyP (generic function with 1 method)

In [17]:
(wT,muT,StdT) = MVTangencyP(μ,Σ,Rf)
println("Tangency portfolio: ")
printmat(wT)
printlnPs("mean and std of tangency portfolio, %: ",[muT StdT]*100)

Tangency portfolio: 
     0.248
     0.682
     0.070

mean and std of tangency portfolio, %:      9.750     7.372


By mixing the tangency portfolio and the riskfree, we can create any point on the MV frontier of risky and riskfree (also called the Capital Market Line, CML). 

The code below shows the expected return and standard deviation of several portfolio (different $v$ values) of the form

$R_p = v R_T + (1-v)R_f$ where $R_T=w_T'R$

In [21]:
v = [0;0.44;1;1.41]                                 #try different mixes of wT and Rf

(ERLev,StdLev)  = (similar(v),similar(v))           #to store results in
for i = 1:length(v)                    #loop over different v values (mix of wT and Rf)
    ERLev[i]   = v[i]'muT + (1-v[i])*Rf     #portfolio with v[i] in wT and 1-v[i] in Rf
    StdLev[i]  = abs(v[i])*StdT
end

In [22]:
plot(StdRp*100,mustar*100,legend=nothing,color=:magenta,xlim=(0,15),ylim=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(StdLev*100,ERLev*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip8902)" style="stroke:#ff00ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 503.328,294.396 495.201,291.767 487.126,289.138 479.103,286.509 471.137,283.88 463.23,281.251 455.386,278.622 447.608,275.993 439.901,273.364 432.268,270.735 
 424.715,268.106 417.246,265.477 409.865,262.848 402.58,260.219 395.395,257.59 388.316,254.961 381.352,252.332 374.508,249.703 367.793,247.074 361.214,244.445 
 354.781,241.816 348.502,239.187 342.387,236.558 336.448,233.929 330.694,231.3 325.137,228.671 319.789,226.042 314.662,223.413 309.77,220.784 305.126,218.155 
 300.742,215.526 296.633,212.897 292.812,210.268 289.293,207.639 286.089,205.01 283.213,202.381 280.676,199.752 278.49,197.123 276.665,194.494 275.209,191.865 
 274.13,189.236 273.432,186.607 273.119,183.978 273.193,181.349 273.653,178.72 274.497,176.091 275.721,173.462 277.319,170.833 279.283,168.204 281.604,165.575 
 284.272,162.946 287.275,160.317 290.601,157.688 294.237,155.059 298.169,152.43 302.385,149.801 306.87,147.172 311.611,144.543 316.594,141.914 321.807,139.285 
 327.236,136.656 332.87,134.027 338.696,131.398 344.704,128.769 350.882,126.14 357.221,123.511 363.711,120.882 370.343,118.253 377.108,115.624 383.999,112.995 
 391.008,110.366 398.128,107.737 405.352,105.108 412.674,102.479 420.089,99.85 427.591,97.221 435.176,94.592 442.837,91.963 450.572,89.334 458.375,86.705 
 466.244,84.076 474.174,81.447 482.162,78.818 490.205,76.189 498.301,73.56 506.445,70.931 514.637,68.302 522.873,65.673 531.151,63.044 539.469,60.415 
 547.825,57.786 556.217,55.157 564.644,52.528 573.103,49.899 581.594,47.27 590.114,44.641 598.663,42.012 607.239,39.3831 615.841,36.7541 624.467,34.1251 
 633.117,31.4961 
 "/>
<polyline clip-path="url(#clip8902)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8772,294.396 49.5555,291.767 54.2338,289.138 58.9121,286.509 63.5905,283.88 68.2688,281.251 72.9471,278.622 77.6254,275.993 82.3037,273.364 86.982,270.735 
 91.6603,268.106 96.3386,265.477 101.017,262.848 105.695,260.219 110.374,257.59 115.052,254.961 119.73,252.332 124.408,249.703 129.087,247.074 133.765,244.445 
 138.443,241.816 143.122,239.187 147.8,236.558 152.478,233.929 157.157,231.3 161.835,228.671 166.513,226.042 171.191,223.413 175.87,220.784 180.548,218.155 
 185.226,215.526 189.905,212.897 194.583,210.268 199.261,207.639 203.94,205.01 208.618,202.381 213.296,199.752 217.975,197.123 222.653,194.494 227.331,191.865 
 232.009,189.236 236.688,186.607 241.366,183.978 246.044,181.349 250.723,178.72 255.401,176.091 260.079,173.462 264.758,170.833 269.436,168.204 274.114,165.575 
 278.792,162.946 283.471,160.317 288.149,157.688 292.827,155.059 297.506,152.43 302.184,149.801 306.862,147.172 311.541,144.543 316.219,141.914 320.897,139.285 
 325.576,136.656 330.254,134.027 334.932,131.398 339.61,128.769 344.289,126.14 348.967,123.511 353.645,120.882 358.324,118.253 363.002,115.624 367.68,112.995 
 372.359,110.366 377.037,107.737 381.715,105.108 386.394,102.479 391.072,99.85 395.75,97.221 400.428,94.592 405.107,91.963 409.785,89.334 414.463,86.705 
 419.142,84.076 423.82,81.447 428.498,78.818 433.177,76.189 437.855,73.56 442.533,70.931 447.211,68.302 451.89,65.673 456.568,63.044 461.246,60.415 
 465.925,57.786 470.603,55.157 475.281,52.528 479.96,49.899 484.638,47.27 489.316,44.641 493.995,42.012 498.673,39.3831 503.351,36.7541 508.029,34.1251 
 512.708,31.4961 
 "/>

## Examples of Tangency Portfolios

In [23]:
μb = [9; 6]/100                     #means
Σb = [ 256  0;
      0    144]/100^2
Rfb = 1/100
wT, = MVTangencyP(μb,Σb,Rfb)
printmat(wT)

wT, = MVTangencyP([13; 6]/100,Σb,Rfb)
printmat(wT)

Σb = [ 1  -0.8;
      -0.8    1]
wT, = MVTangencyP(μb,Σb,Rfb)
printmat(wT)

Σb = [ 1  0.8;
      0.8    1]
wT, = MVTangencyP(μb,Σb,Rfb)
printmat(wT)

     0.474
     0.526

     0.574
     0.426

     0.513
     0.487

     1.538
    -0.538



# Drawing the MV Frontier Revisisted (extra)

Recall: with only two investable assets, all portfolios of them are on the MV frontier.

We apply this idea by using the global minimum variance portfolio (see below for code) and the tangency portfolios (see above).

In [24]:
function MVMinimumVarP(μ,Σ,Rf)           #calculates the global minimum variance portfolio
    n     = length(μ)
    oneV  = ones(n)  
    μe    = μ .- Rf                     
    Σ_1   = inv(Σ)
    w     = Σ_1*oneV/(oneV'Σ_1*oneV)
    mu    = w'μ + (1-sum(w))*Rf
    Std   = sqrt(w'Σ*w)
    return w,mu,Std
end

MVMinimumVarP (generic function with 1 method)

In [25]:
(wT,muT,StdT) = MVTangencyP(μ,Σ,Rf)
(wMvp,muMvp,StdMvp) = MVMinimumVarP(μ,Σ,Rf)
println("Tangency and minum variance portfolios: ")
printmat([wT wMvp])

Tangency and minum variance portfolios: 
     0.248    -0.020
     0.682     0.625
     0.070     0.395



In [26]:
v = linspace(-2.5,2.5,101)                                 #try different mixes of wT and wMvp

(ERLev,StdLev)  = (similar(v),similar(v))
for i = 1:length(v)
    local w
    w          = v[i]*wT + (1-v[i])*wMvp                        #portfolio with v[i] in wT and 1-v[i] in wMvp
    ERLev[i]   = w'μ
    StdLev[i]  = sqrt(w'Σ*w)
end

In [27]:
plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlim=(0,15),ylim=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(StdLev*100,ERLev*100)
title!("Mean vs standard deviation: scatter points are new")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 Mean vs standard deviation: scatter points are new 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip9102)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 503.328,294.396 495.201,291.767 487.126,289.138 479.103,286.509 471.137,283.88 463.23,281.251 455.386,278.622 447.608,275.993 439.901,273.364 432.268,270.735 
 424.715,268.106 417.246,265.477 409.865,262.848 402.58,260.219 395.395,257.59 388.316,254.961 381.352,252.332 374.508,249.703 367.793,247.074 361.214,244.445 
 354.781,241.816 348.502,239.187 342.387,236.558 336.448,233.929 330.694,231.3 325.137,228.671 319.789,226.042 314.662,223.413 309.77,220.784 305.126,218.155 
 300.742,215.526 296.633,212.897 292.812,210.268 289.293,207.639 286.089,205.01 283.213,202.381 280.676,199.752 278.49,197.123 276.665,194.494 275.209,191.865 
 274.13,189.236 273.432,186.607 273.119,183.978 273.193,181.349 273.653,178.72 274.497,176.091 275.721,173.462 277.319,170.833 279.283,168.204 281.604,165.575 
 284.272,162.946 287.275,160.317 290.601,157.688 294.237,155.059 298.169,152.43 302.385,149.801 306.87,147.172 311.611,144.543 316.594,141.914 321.807,139.285 
 327.236,136.656 332.87,134.027 338.696,131.398 344.704,128.769 350.882,126.14 357.221,123.511 363.711,120.882 370.343,118.253 377.108,115.624 383.999,112.995 
 391.008,110.366 398.128,107.737 405.352,105.108 412.674,102.479 420.089,99.85 427.591,97.221 435.176,94.592 442.837,91.963 450.572,89.334 458.375,86.705 
 466.244,84.076 474.174,81.447 482.162,78.818 490.205,76.189 498.301,73.56 506.445,70.931 514.637,68.302 522.873,65.673 531.151,63.044 539.469,60.415 
 547.825,57.786 556.217,55.157 564.644,52.528 573.103,49.899 581.594,47.27 590.114,44.641 598.663,42.012 607.239,39.3831 615.841,36.7541 624.467,34.1251 
 633.117,31.4961 
 "/>
<polyline clip-path="url(#clip9102)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8772,294.396 49.5555,291.767 54.2338,289.138 58.9121,286.509 63.5905,283.88 68.2688,281.251 72.9471,278.622 77.6254,275.993 82.3037,273.364 86.982,270.735 
 91.6603,268.106 96.3386,265.477 101.017,262.848 105.695,260.219 110.374,257.59 115.052,254.961 119.73,252.332 124.408,249.703 129.087,247.074 133.765,244.445 
 138.443,241.816 143.122,239.187 147.8,236.558 152.478,233.929 157.157,231.3 161.835,228.671 166.513,226.042 171.191,223.413 175.87,220.784 180.548,218.155 
 185.226,215.526 189.905,212.897 194.583,210.268 199.261,207.639 203.94,205.01 208.618,202.381 213.296,199.752 217.975,197.123 222.653,194.494 227.331,191.865 
 232.009,189.236 236.688,186.607 241.366,183.978 246.044,181.349 250.723,178.72 255.401,176.091 260.079,173.462 264.758,170.833 269.436,168.204 274.114,165.575 
 278.792,162.946 283.471,160.317 288.149,157.688 292.827,155.059 297.506,152.43 302.184,149.801 306.862,147.172 311.541,144.543 316.219,141.914 320.897,139.285 
 325.576,136.656 330.254,134.027 334.932,131.398 339.61,128.769 344.289,126.14 348.967,123.511 353.645,120.882 358.324,118.253 363.002,115.624 367.68,112.995 
 372.359,110.366 377.037,107.737 381.715,105.108 386.394,102.479 391.072,99.85 395.75,97.221 400.428,94.592 405.107,91.963 409.785,89.334 414.463,86.705 
 419.142,84.076 423.82,81.447 428.498,78.818 433.177,76.189 437.855,73.56 442.533,70.931 447.211,68.302 451.89,65.673 456.568,63.044 461.246,60.415 
 465.925,57.786 470.603,55.157 475.281,52.528 479.96,49.899 484.638,47.27 489.316,44.641 493.995,42.012 498.673,39.3831 503.351,36.7541 508.029,34.1251 
 512.708,31.4961 
 "/>